# TASK

Write a simulation for  a simple game of chance (e.g., drawing cards, rolling one or more dice, etc.)   Run the simulation 1000 times and interpret the outcome.  Post your code here, and discuss what might be done to improve it.

# INTRODUCTION

In the game Magic: the Gathering, each player begins with 20 life. Players lose life when cards deal "damage" to them, losing one point of life for each point of damage that they take. If one player reduces the other to 0 life, they win. There is a caveat however, the winning player must remain "alive" (above 0 life). If both players simultaneously reach 0 or less life, the game is a draw, _even if one player is at a lower life total than the other_ (such as one player at -5 life and one at 0, this game would still be a draw)! In order to add strategic depth to the game, more powerful cards with a bigger impact on the game cannot be played until later turns.

# MANA CLASH

The card Mana Clash was first printed as part of the "The Dark" expansion of Magic: the Gathering in 1994. It was subsequently reprinted in the sets Fourth Edition (1995), Fifth Edition (1997), Seventh Edition (2001), Eighth Edition (2003), and Ninth Edition (2005)$^1$. The card costs only one "mana," meaning it can be played on the first turn. The "Oracle Text" (Magic: the Gathering's term for official rules text, to account for minor variations in wording between editions) for the card is as follows:

> You and target opponent each flip a coin. Mana Clash deals 1 damage to each player whose coin comes up tails. Repeat this process until both players' coins come up heads on the same flip.$^2$

Because players do not stop flipping until both flip heads at the same time, there is no limit to how much damage a player can take from this card. A player who is very lucky could cast it on the first turn, watch their opponent flip 20 consecutive tails, and win the game.

The card was discussed in the June 9th, 2023 episode of The Resleevables podcast$^3$, hosted by Cedric Phillips and Patrick Sullivan:

>**Phillips:** Mana Clash is the only card with a casting cost of a single red that can potentially win the game on the first turn without any other assistance.<br><br>**Sullivan:** Well, the odds of that are quite low.<br><br>**Phillips:** Someone...you know what, I'm coming to our YouTube audience right now. I'm gonna go to my direct cam, right here. I want the math in a YouTube comment, or tweet, or something.<br><br>**Sullivan:** So, the math has to be you win...you specifically win. Because part of the range of this card is also _you_ die on the first turn. Most of the time what happens with this is both players take a relatively small amount of damage and then you move on with life. It is technically possible, like fractions of fractions of fractions of fractions of a percent, for this to generate a turn one kill. But it's not that it always...it kills you with the same frequency it kills your opponent.<br><br>**Phillips:** Very rarely, is what I would say.<br><br>**Sullivan:** But yeah, I'm curious of the math.<br><br>**Phillips:** Yeah, if there's a, if there's a, you just called it a mathemagician? Is that what you said earlier?<br><br>**Sullivan:** Yeah, a mathemagician.<br><br>**Phillips:** Yeah if there's a mathemagician watching the show, I'd love to know what insane math is necessary for someone to get killed by this.<br><br>**Sullivan:** Well it's complicated because there are _so_ many different permutations of the way that it can kill on turn one.<br><br>**Phillips:** I know.<br><br>**Sullivan:** It's like, I could just...you lose twenty flips and I don't lose any, I lose one, I lose two, we each lose nineteen then you lose the twentieth, I lose twelve flips you lose twenty, like there's just a lot of-<br><br>**Phillips:** I'm sure that someone knows out there.<br><br>**Sullivan:** -it's not simple math.<br><br>**Phillips:** No. Not for us. But maybe for you.

Patrick Sullivan is right that _calculating_ the probability of a turn-one Mana Clash ending the game is very difficult. Fortunately, we can simulate the outcome instead.

---

$^1$The list of sets with a printing of Mana Clash can be found [here](https://gatherer.wizards.com/Pages/Card/Details.aspx?multiverseid=1798), and the list of release dates for those sets [here](https://en.wikipedia.org/wiki/List_of_Magic:_The_Gathering_sets).

$^2$The oracle text for Mana Clash can be found [here](https://gatherer.wizards.com/Pages/Card/Details.aspx?multiverseid=1798).

$^3$The podcast video can be found [here](https://www.youtube.com/watch?v=-oS8tY-rFK8). The timestamp for the transcribed portion is 57:46.


#SIMULATION

When a first-turn Mana Clash resolves$^1$, there are four possible outcomes:


1. The caster (Player A) wins the game because the target (Player B) is reduced to 0 or less life while Player A is not.
2. The caster (Player A) loses the game because they are reduced to 0 or less life while the target (Player B) is not.
3. The game ends in a draw because both players are reduced to 0 or less life.
4. The game continues because both players have a life total greater than 0.


To estimate the probability of each of these outcomes experimentally, we can simulate the resolution of a first-turn Mana Clash many times, and determine in what percentage of the total simulations each outcomes occurs. Below, this is simulated 1,000 times.

$^1$There are a number of ways that a player's life total might change in ths first turn for other reasons before Mana Clash resolves. For the purposes of this analysis, assume that both players have an unchanged starting life total of 20 and no other effects are present.

## ONE THOUSAND TRIALS

In [74]:
import random
import pandas as pd

random.seed(31415)

#This function simulates the resolution of the card mana clash and returns the resulting game state
def mana_clash():

  #Each player's life total starts at 20
  p1_life = 20
  p2_life = 20

  #Coin flips will be simulated by random numbers, with 0 representing a tails and 1 representing a heads
  p1_flip = 0
  p2_flip = 0

  #As long as one player flipped tails, the players keep flipping
  while p1_flip + p2_flip != 2:
    p1_flip = random.randint(0, 1)
    p2_flip = random.randint(0, 1)

    #Each player whose flip comes up tails loses one life
    if p1_flip == 0: p1_life -= 1
    if p2_flip == 0: p2_life -= 1

  '''
  If one player's life total is 0 or less and the other's is not, that player loses.
  If both players' life totals are 0 or less, the game ends in a draw.
  Otherwise, the game isn't over and play continues (but the card Mana Clash is discarded).
  '''
  if p1_life <= 0 and p2_life > 0: return "Winner"
  elif p2_life <= 0 and p1_life > 0: return "Winner"
  elif p1_life <= 0 and p2_life <= 0: return "Draw"
  else: return "Ongoing"

#This loop simulates 1,000 resolutions of Mana Clash
clash_results = [mana_clash() for i in range(1000)]

#Determine results
result = ["Winner", "Draw", "Ongoing"]
count = [clash_results.count(result[i]) for i in range(len(result))]
probability = [count[i]/len(clash_results) for i in range(len(result))]

#Print results table
results_frame = pd.DataFrame(data = {"Outcome": result, "Count": count, "Probability": probability})
print(results_frame)

   Outcome  Count  Probability
0   Winner      0          0.0
1     Draw      0          0.0
2  Ongoing   1000          1.0


In every trial, the game continues after the resolution of Mana Clash. The experimental probability of this outcome is 1.0, and the experimental probability of all other outcomes is 0.

## TEN MILLION TRIALS

The results of the previous simulation with 1,000 trials show that the probability of a player winning as a result of a first-turn Mana Clash is very small, too small to get an accurate estimate from 1,000 trials. A larger number of trials are needed to estimate very small probabilities by experimentation. Below, we try with 10,000,000 trials.

In [71]:
#This loop simulates 10,000,000 resolutions of Mana Clash
clash_longer = [mana_clash() for i in range(10000000)]

#Determine results
result_2 = ["Winner", "Draw", "Ongoing"]
count_2 = [clash_longer.count(result_2[i]) for i in range(len(result_2))]
probability_2 = [count_2[i]/len(clash_longer) for i in range(len(result_2))]

#Print results table
results_frame_2 = pd.DataFrame(data = {"Outcome": result_2, "Count": count_2, "Probability": probability_2})
print(results_frame_2)

   Outcome    Count  Probability
0   Winner     3674     0.000367
1     Draw     1159     0.000116
2  Ongoing  9995167     0.999517


In 9,995,167 of the 10,000,000 trials, the game continues. The experimental probability of this outcome is 0.9995167, about 99.95%. The game ends in a draw in 1,159 trials, so this outcome has an experimental probability of 0.0001159, or about 0.01%. There are 3,674 trials where a first-turn Mana Clash results in a player winning the game, so this outcome has an experimental probability of 0.0003674, or about 0.04%. Since the likelihood of each player winning is the same, the experimental probability of a specific player winning the game is 0.0001837, or about 0.02%.

#CONFIDENCE INTERVAL

From the simulation results, it is possible to calculate a confidence interval for the true probability (true population proportion) that a first-turn Mana Clash will result in a player winning the game. I'm sure that there's a package somewhere that could calculate this, but I wanted to practice working with simple Python functions a little bit more, so I wrote my own function to calculate the confidence interval below.

The confidence interval can be calculated as:

$\hat{p}\pm 1.96\times \sqrt{\frac{\hat{p}(1-\hat{p})}{n}}$

In [72]:
import numpy as np

def confidence_interval(p, n):
  se_p_hat = np.sqrt((p * (1 - p) / n))
  low_ci = p - (1.96 * se_p_hat)
  high_ci = p + (1.96 * se_p_hat)
  return low_ci, high_ci

confidence_interval(probability_2[0], 10000000)


(0.0003555219307535728, 0.00037927806924642715)

Based on this calculation, we can be 95% confident that the true probability of a first-turn Mana Clash resulting in a player winning the game is between 0.035552% and 0.03793%. Each individual player's probability of winning is then between 0.017776% and 0.018965%. This level of precision is not likely to be necessary. We can safely summarize these conclusions by saying that if you cast a first-turn Mana Clash, your probability of winning the game with it is between 0.018% and 0.019%, or just under 0.02% (a chance of about 1 in 5,000).

# ALTERNATE CODE

A second method of simulating the resolution of Mana Clash is to use random.choice to select between Heads and Tails for the coin flip rather than random.randint. This method is shown below.

In [73]:
#This function simulates the resolution of the card mana clash and returns the resulting game state
def mana_clash_v2():

  #Each player's life total starts at 20
  p1_life = 20
  p2_life = 20

  #Coin flips will be simulated by a random selection of either "Heads" or "Tails"
  flip_results = ["Heads", "Tails"]
  p1_flip = ""
  p2_flip = ""

  #As long as one player flipped tails, the players keep flipping
  while p1_flip != "Heads" or p2_flip != "Heads":
    p1_flip = random.choice(flip_results)
    p2_flip = random.choice(flip_results)

    #Each player whose flip comes up tails loses one life
    if p1_flip == "Tails": p1_life -= 1
    if p2_flip == "Tails": p2_life -= 1

  '''
  If one player's life total is 0 or less and the other's is not, that player loses.
  If both players' life totals are 0 or less, the game ends in a draw.
  Otherwise, the game isn't over and play continues (but the card Mana Clash is discarded).
  '''
  if p1_life <= 0 and p2_life > 0: return "Winner"
  elif p2_life <= 0 and p1_life > 0: return "Winner"
  elif p1_life <= 0 and p2_life <= 0: return "Draw"
  else: return "Ongoing"

#This loop simulates 10,000,000 resolutions of Mana Clash
clash_results_alt = [mana_clash_v2() for i in range(10000000)]

#Determine results
result_alt = ["Winner", "Draw", "Ongoing"]
count_alt = [clash_results_alt.count(result_alt[i]) for i in range(len(result_alt))]
probability_alt = [count_alt[i]/len(clash_results_alt) for i in range(len(result_alt))]

#Print results table
results_frame_alt = pd.DataFrame(data = {"Outcome": result_alt, "Count": count_alt, "Probability": probability_alt})
print(results_frame_alt)

   Outcome    Count  Probability
0   Winner     3615     0.000362
1     Draw     1189     0.000119
2  Ongoing  9995196     0.999520
